In [1]:
# 四半期の騰落率が大きく動いた銘柄の過去分析行う
# 　四半期騰落率過去分析1: すでにある程度業績の良い銘柄の分析
# 　四半期騰落率過去分析2: 黒字化銘柄の分析

In [2]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import KessanPl, PricelistFig, PricelistPl, MeigaralistPl, read_data
from lib_dataprocess import ShikihoPl, FinancequotePl, print_finance_quote, PricelistFig, KessanFig, IndexPricelistPl
import polars as pl
from datetime import date
from dateutil.relativedelta import relativedelta

RevPl = PricelistPl("reviced_pricelist.parquet")
RawPL = PricelistPl("raw_pricelist.parquet")
MPL = MeigaralistPl()
SPL = ShikihoPl()

In [3]:
#####
##### 評価日における四半期騰落率一覧
#####

In [4]:
valuation_date = date(2024, 8, 1)


KPl = KessanPl()
updown_rate_df = KPl.get_quater_settlement_price_updown_rate(valuation_date, "nh225")
updown_rate_df

code,start_date,end_date,updown_rate,nh_updown_rate
i64,date,date,f64,f64
1301,2024-05-11,2024-08-05,-8.97,-17.67
1332,2024-05-15,2024-08-06,-12.67,-10.01
1333,2024-05-08,2024-08-05,-11.23,-18.67
1375,2024-05-10,2024-08-08,-0.41,-9.2
1376,2024-07-11,2024-10-04,-3.01,-8.76
…,…,…,…,…
9993,2024-07-12,2024-10-10,-5.32,-5.49
9994,2024-05-16,2024-08-09,-3.61,-9.37
9995,2024-05-08,2024-08-01,0.27,-1.43


In [5]:
#####
##### valuation_date時点で発表済四半期決算の業績成長率一覧(growth_rate_df)の作成
#####

In [6]:
KPL1 = KessanPl()
KPL2 = KessanPl()
KPL1.with_columns_growthrate_lastyear()
KPL2.with_columns_diff_growthrate()
KPL1.df = KPL1.df.filter(pl.col("settlement_type")=="四")
KPL2.df = KPL2.df.filter(pl.col("settlement_type")=="四")
df = KPL1.df.join(KPL2.df, on=["code", "settlement_date"], how="left")
KPL3 = KessanPl(df)
df =KPL3.get_latest_quater_settlements(valuation_date)
df = df.select([
    "code", "settlement_date", "announcement_date", "sales", "operating_income", "sales_growthrate", "diff_operating_income_growthrate"
])
df = df.rename({"settlement_date": "評価日発表済最新四半期決算"})
growth_rate_df = df
growth_rate_df

code,評価日発表済最新四半期決算,announcement_date,sales,operating_income,sales_growthrate,diff_operating_income_growthrate
i64,date,date,i64,i64,f64,f64
1301,2024-03-31,2024-05-10,59990,1857,2.1,155.0
1332,2024-03-31,2024-05-14,205894,3308,8.5,13.6
1333,2024-03-31,2024-05-07,243328,1176,1.6,-37.3
1375,2024-03-31,2024-05-09,11807,-803,21.1,-5.0
1376,2024-05-31,2024-07-10,23224,1150,1.9,-16.0
…,…,…,…,…,…,…
9993,2024-05-31,2024-07-11,24969,-408,1.8,-108.5
9994,2024-03-31,2024-05-15,36543,1082,3.1,63.8
9995,2024-03-31,2024-05-07,18708,63,7.7,0.1


In [7]:
#####
##### valuation_date時点の最新financial_quote(finance_df)の抽出
#####
FPL = FinancequotePl()
FPL.with_columns_market_cap()
FPL.with_columns_ROA()
FPL.with_columns_ROE()
df = FPL.get_finance_quotes(valuation_date)
df = df.select([
    "code",
    "date",
    "expected_PER",
    "actual_PBR",
    "actual_CAR",
    "market_cap",
    "ROA",
    "ROE"
])

finance_df = df
finance_df

code,date,expected_PER,actual_PBR,actual_CAR,market_cap,ROA,ROE
i64,date,f64,f64,f64,i64,f64,f64
1301,2024-08-01,6.66,0.79,36.7,47407,4.36,11.87
1332,2024-08-01,11.26,1.08,41.1,271564,3.96,9.63
1333,2024-08-01,8.64,0.79,30.8,164886,2.83,9.17
1375,2024-08-01,25.74,3.48,30.1,39910,4.07,13.53
1376,2024-08-01,13.23,0.65,50.2,16540,2.46,4.91
…,…,…,…,…,…,…,…
9991,2024-08-01,7.95,0.52,58.8,32579,3.82,6.49
9993,2024-08-01,111.58,0.47,49.6,13624,0.21,0.42
9994,2024-08-01,9.05,1.04,51.6,33520,5.93,11.49


In [8]:
#####
##### 評価dfの作成：　updown_rate_dfとgrowth_rate_dfをjoinして必要な列を残す
#####

In [9]:
df = growth_rate_df.join(updown_rate_df, on=["code"], how="left")
if finance_df.shape[0] > 3500:
    df = df.join(finance_df, on=["code"], how="left")
    
df = df.with_columns([
    (pl.lit(100) * pl.col("operating_income") / pl.col("sales")).round(2).alias("営業利益率")
])
df = df.select(df.columns[:5] + [df.columns[-1]] + df.columns[5:-1])
df = df.select(df.columns[:2]+df.columns[3:])
df = df.sort(by=["updown_rate"], descending=[True])
df = df.drop_nulls()
df = df.sort(by=["code"])
evaluated_df = df
evaluated_df

code,評価日発表済最新四半期決算,sales,operating_income,営業利益率,sales_growthrate,diff_operating_income_growthrate,start_date,end_date,updown_rate,nh_updown_rate,date,expected_PER,actual_PBR,actual_CAR,market_cap,ROA,ROE
i64,date,i64,i64,f64,f64,f64,date,date,f64,f64,date,f64,f64,f64,i64,f64,f64
1301,2024-03-31,59990,1857,3.1,2.1,155.0,2024-05-11,2024-08-05,-8.97,-17.67,2024-08-01,6.66,0.79,36.7,47407,4.36,11.87
1332,2024-03-31,205894,3308,1.61,8.5,13.6,2024-05-15,2024-08-06,-12.67,-10.01,2024-08-01,11.26,1.08,41.1,271564,3.96,9.63
1333,2024-03-31,243328,1176,0.48,1.6,-37.3,2024-05-08,2024-08-05,-11.23,-18.67,2024-08-01,8.64,0.79,30.8,164886,2.83,9.17
1375,2024-03-31,11807,-803,-6.8,21.1,-5.0,2024-05-10,2024-08-08,-0.41,-9.2,2024-08-01,25.74,3.48,30.1,39910,4.07,13.53
1376,2024-05-31,23224,1150,4.95,1.9,-16.0,2024-07-11,2024-10-04,-3.01,-8.76,2024-08-01,13.23,0.65,50.2,16540,2.46,4.91
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9991,2024-06-30,26954,1366,5.07,-14.5,-2.5,2024-07-26,2024-10-29,-9.93,2.94,2024-08-01,7.95,0.52,58.8,32579,3.82,6.49
9993,2024-05-31,24969,-408,-1.63,1.8,-108.5,2024-07-12,2024-10-10,-5.32,-5.49,2024-08-01,111.58,0.47,49.6,13624,0.21,0.42
9994,2024-03-31,36543,1082,2.96,3.1,63.8,2024-05-16,2024-08-09,-3.61,-9.37,2024-08-01,9.05,1.04,51.6,33520,5.93,11.49


In [10]:
#####
##### filter
#####

In [16]:
# para
# 業績para
finance_filter = False

min_sales = 5000

min_operating_income = -10000000

min_営業利益率 = -100
max_営業利益率 = 0

min_sales_growth_rate = 10
max_sales_growth_rate = 100

min_diff_operating_income_growthrate = 0

r1 = 0
# 財務para
max_expected_PER = 100 
max_actual_PBR = 10
min_actual_CAR = 50
min_ROA = -100
min_ROE = -100


# 単純filter
df = evaluated_df
# 業績filter
df = df.filter(pl.col("sales")>=min_sales)\
    .filter(pl.col("operating_income")>=min_operating_income)\
    .filter(pl.col("営業利益率")>=min_営業利益率)\
    .filter(pl.col("営業利益率")<=max_営業利益率)\
    .filter(pl.col("sales_growthrate")>=min_sales_growth_rate)\
    .filter(pl.col("sales_growthrate")<=max_sales_growth_rate)\
    .filter(pl.col("diff_operating_income_growthrate")>=min_diff_operating_income_growthrate)

# 財務filter
# 黒字化企業をfilterする場合は、実行しない
if finance_filter:
    df = df.filter(pl.col("expected_PER")<=max_expected_PER)\
        .filter(pl.col("actual_PBR")<=max_actual_PBR)\
        .filter(pl.col("actual_CAR")>=min_actual_CAR)\
        .filter(pl.col("ROA")>=min_ROA)\
        .filter(pl.col("ROE")>=min_ROE)

# 列間filter
df = df.filter(
    pl.lit(r1) * pl.col("sales_growthrate") <= pl.col("diff_operating_income_growthrate")
)

target_df = df
target_df


code,評価日発表済最新四半期決算,sales,operating_income,営業利益率,sales_growthrate,diff_operating_income_growthrate,start_date,end_date,updown_rate,nh_updown_rate,date,expected_PER,actual_PBR,actual_CAR,market_cap,ROA,ROE
i64,date,i64,i64,f64,f64,f64,date,date,f64,f64,date,f64,f64,f64,i64,f64,f64
1815,2024-03-31,50928,-3,-0.01,11.3,4.0,2024-05-15,2024-08-09,-11.53,-9.1,2024-08-01,13.86,0.52,34.0,39611,1.28,3.76
1950,2024-06-30,33011,-774,-2.34,15.5,11.3,2024-08-01,2024-10-31,-1.57,0.77,2024-08-01,10.29,0.59,65.7,112797,3.78,5.75
2130,2024-06-30,5035,-492,-9.77,11.9,13.0,2024-08-01,2024-10-31,-1.94,0.77,2024-08-01,73.36,2.11,51.5,11569,1.48,2.87
5261,2024-03-31,5910,-45,-0.76,30.9,42.0,2024-05-10,2024-08-09,-13.58,-8.7,2024-08-01,17.24,1.77,34.0,25901,3.48,10.25
9033,2024-03-31,9070,-660,-7.28,22.7,21.9,2024-05-15,2024-08-09,-4.05,-9.1,2024-08-01,26.21,0.54,41.3,22072,0.85,2.07
9206,2024-06-30,10068,-854,-8.48,13.9,105.7,2024-08-01,2024-10-31,-2.5,0.77,2024-08-01,6.88,3.6,13.6,9595,7.11,52.31
9474,2024-06-30,14130,-274,-1.94,10.5,80.0,2024-07-30,2024-10-29,-19.1,1.73,2024-08-01,20.3,1.02,65.3,54493,3.27,5.01
9551,2024-06-30,25015,-2068,-8.27,18.2,11.8,2024-07-26,2024-10-30,-3.46,3.93,2024-08-01,13.26,1.13,43.2,84836,3.69,8.55
9850,2024-03-31,8884,-166,-1.87,24.5,13.2,2024-05-15,2024-08-09,0.19,-9.1,2024-08-01,118.62,2.92,23.2,25544,0.57,2.46


In [17]:
evaluated_df

code,評価日発表済最新四半期決算,sales,operating_income,営業利益率,sales_growthrate,diff_operating_income_growthrate,start_date,end_date,updown_rate,nh_updown_rate,date,expected_PER,actual_PBR,actual_CAR,market_cap,ROA,ROE
i64,date,i64,i64,f64,f64,f64,date,date,f64,f64,date,f64,f64,f64,i64,f64,f64
1301,2024-03-31,59990,1857,3.1,2.1,155.0,2024-05-11,2024-08-05,-8.97,-17.67,2024-08-01,6.66,0.79,36.7,47407,4.36,11.87
1332,2024-03-31,205894,3308,1.61,8.5,13.6,2024-05-15,2024-08-06,-12.67,-10.01,2024-08-01,11.26,1.08,41.1,271564,3.96,9.63
1333,2024-03-31,243328,1176,0.48,1.6,-37.3,2024-05-08,2024-08-05,-11.23,-18.67,2024-08-01,8.64,0.79,30.8,164886,2.83,9.17
1375,2024-03-31,11807,-803,-6.8,21.1,-5.0,2024-05-10,2024-08-08,-0.41,-9.2,2024-08-01,25.74,3.48,30.1,39910,4.07,13.53
1376,2024-05-31,23224,1150,4.95,1.9,-16.0,2024-07-11,2024-10-04,-3.01,-8.76,2024-08-01,13.23,0.65,50.2,16540,2.46,4.91
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9991,2024-06-30,26954,1366,5.07,-14.5,-2.5,2024-07-26,2024-10-29,-9.93,2.94,2024-08-01,7.95,0.52,58.8,32579,3.82,6.49
9993,2024-05-31,24969,-408,-1.63,1.8,-108.5,2024-07-12,2024-10-10,-5.32,-5.49,2024-08-01,111.58,0.47,49.6,13624,0.21,0.42
9994,2024-03-31,36543,1082,2.96,3.1,63.8,2024-05-16,2024-08-09,-3.61,-9.37,2024-08-01,9.05,1.04,51.6,33520,5.93,11.49


In [18]:
#####
##### 統計結果出力
#####

In [19]:
df = evaluated_df
print("##### 全体")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'全銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')
print()

df = target_df
print("##### 抽出銘柄")
print(f'総銘柄数：{df.shape[0]}')
a = df.filter(pl.col("updown_rate")>0).shape[0]
b = df.filter(pl.col("updown_rate")<=0).shape[0]
uprate = round(100*a/(a+b), 1)
print(f'上昇銘柄数:{a}／下落銘柄数:{b}／上昇率:({uprate}%)')
print(f'抽出銘柄平均騰落率:{round(df["updown_rate"].mean(), 2)}%')
print(f'日経225平均騰落率:{round(df["nh_updown_rate"].mean(), 2)}%')

##### 全体
総銘柄数：3241
上昇銘柄数:789／下落銘柄数:2452／上昇率:(24.3%)
全銘柄平均騰落率:-6.39%
日経225平均騰落率:-6.19%

##### 抽出銘柄
総銘柄数：9
上昇銘柄数:1／下落銘柄数:8／上昇率:(11.1%)
抽出銘柄平均騰落率:-6.39%
日経225平均騰落率:-3.11%


In [20]:
#####
##### 個別銘柄の分析
#####

In [21]:
# ファンダメンタルズ
code = 7254
FPL.print_finance_info(code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)



ユニバンス(7254)の銘柄情報

終値: 488.0円(2024年08月01日)
予想配当利回り: 2.46%(2024年08月01日)
予想PER: 4.07倍(2024年08月01日)
実績PBR: 0.41倍(2024年08月01日)
自己資本比率: 53.2%(2024年08月01日)
予想ROE: 10.14%(2024年08月01日)
予想ROA: 5.39%(2024年08月01日)
時価総額: 114.2億円(2024年08月01日)

7254(ユニバンス)の四季報データ履歴

発行日: 2024年06月14日
【反落】
  主力の駆動装置は好採算の北米農機向けが増勢。自動車向けは顧客の自動車生産復調につれて伸びる。価格転嫁の進展も効く。だが、製品保証引当金の戻りが剥落。開発費かさみ、人件費も増加。営業益反落。土壌汚染対策特損ない。連続増配。
【開発】
  新規分野の開拓に注力。EV向け駆動装置のほか、ロボット向け歯車など自動車以外の分野への開発・投資を加速。

発行日: 2024年03月16日
【続伸】
  主力の駆動装置は自動車向けが回復。前号より営業益再増額。子会社工場での汚染対策費用計上でも最終増益。増配。25年3月期も主力の駆動装置が好調持続。前期の製品保証引当金なくなっても合理化効果など大きく営業益続伸。大幅増配も。
【ニッチ】
  EV駆動装置は数を追わず超小型EVや大型トラック向けなど特定顧客に応じた開発を志向。他社と差別化し販売。

発行日: 2023年12月15日
【大幅増額】
  主力の駆動装置は半導体不足解消で自動車向けが大幅増。構造改革による固定費や物流費削減に加え、円安効果もあり。営業益表記へ大幅増額。増配。25年3月期も主力の駆動装置が順調、連続増益。
【共同開発】
  独自製作の減速機と一体化したEV駆動装置をヤマハ発動機と開発、EVスーパーカー等の採用目指す。さらにトラックや超小型EV向けの開発も強化。

発行日: 2023年09月15日
【反発】
  主力の駆動装置は大手自動車の半導体不足が解消、販価も向上。前期赤字の米国も最悪期から脱出。非自動車新製品などの立ち上げ負担あるが営業益反発。営業外に為替差益。前期の減損特損消える。
【３つの型】
  独自の電動駆動システムは従来S型にSS型とM型を追加し充実。特に

In [23]:
#####
##### 四半期決算推移
#####

In [24]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [25]:
#####
##### 株価推移
#####

In [26]:
# para
delta = relativedelta(days=100)

In [27]:
# 日足
KPL = KessanPl()
df = KPL.df.filter(pl.col("announcement_date")<valuation_date).filter(pl.col("code")==code)
ann_date = df["announcement_date"].to_list()[-1]
print(f'決算発表日: {ann_date}')
start_date = ann_date - delta
end_date = ann_date + delta

PFIG = PricelistFig(code, RevPl.df, MPL.df, start_date, end_date)
# 決算発表日にvlineを引く
PFIG.add_vline_announcement_date()
PFIG.fig.show()

決算発表日: 2024-05-13
